# Python `map()` — Guía completa

_Notebook generado el 2025-08-10 00:38:44 UTC_.

Este notebook cubre en detalle la función incorporada `map()` de Python 3: comportamiento, ejemplos para cada caso (funciones normales, `lambda`, múltiples iterables, `None`, funciones con efectos secundarios, combinaciones con `itertools`, `operator`, `functools.partial`), comparaciones con comprensiones de lista, rendimiento y buenas prácticas. Está pensado para una competencia nacional — toma en serio los ejemplos, todas las celdas son ejecutables y autocontenidas.

## 1. ¿Qué hace `map()`?

- `map(function, iterable, ...)` aplica `function` a cada elemento de uno o más iterables y devuelve un **objeto iterador** (lazy) en Python 3.
- Si se pasan múltiples iterables, `function` debe aceptar ese número de argumentos; `map` se detiene cuando el iterable más corto se agota.
- No confundir con Python 2 — en Python 2 `map()` devolvía lista (y `map(None, ...)` tenía comportamiento diferente).

In [1]:
# Ejemplo mínimo — map retorna un iterador (lazy) en Python 3
def cuadrado(x):
    return x * x

it = map(cuadrado, [1, 2, 3, 4])
print('map object:', it)
print('Consumir con list():', list(it))
# Al volver a consumir el mismo iterador no habrá elementos
print('Consumir otra vez (vacío):', list(it))


map object: <map object at 0x0000016F3EFCCDC0>
Consumir con list(): [1, 4, 9, 16]
Consumir otra vez (vacío): []


## 2. `map()` con `lambda` — caso muy común


In [ ]:
# Usando lambda para transformar una lista de strings a enteros (trim incluido)
strings = [' 10', '20 ', '  30']
res = map(lambda s: int(s.strip()), strings)
print(list(res))


## 3. Múltiples iterables

- Si pasas varios iterables, la función recibirá argumentos de cada iterable en paralelo.
- `map` termina cuando el iterable más corto se agota.


In [ ]:
a = [1, 2, 3]
b = [10, 20, 30, 40]
def suma(x, y):
    return x + y

print(list(map(suma, a, b)))  # se para en el elemento 3 (más corto es `a`)

# Con lambda
print(list(map(lambda x, y: x * y, a, b)))


## 4. `map()` y evaluación perezosa (lazy evaluation)

- `map()` devuelve un iterador: **no** realiza trabajo hasta que consumes el resultado (ej. `list()`, `for` loop o `next()`).
- Útil para ahorrar memoria en transformaciones grandes.


In [ ]:
import itertools

def triple(x):
    print('procesando', x)
    return x * 3

it = map(triple, range(5))
print('antes de consumir')
print(next(it))  # solo procesa el primer elemento
print('despues de next')
print(list(it))  # consume el resto


## 5. `map()` vs comprensiones de lista

- Comprensiones: más legibles en muchas situaciones, y devuelven listas (en Python 3).
- `map()` útil cuando ya tienes una función nombrada, o quieres un iterador perezoso.

Ejemplo equivalente:


In [ ]:
nums = [1, 2, 3, 4]
def doble(x):
    return 2 * x

print('map:', list(map(doble, nums)))
print('list comp:', [doble(x) for x in nums])

# Cuando la transformacion es simple, la comprension suele ganar en claridad
print('lambda con map:', list(map(lambda x: x + 1, nums)))
print('lambda con list comp:', [x + 1 for x in nums])


## 6. `map()` y funciones con efectos secundarios

- `map()` puede ser usado para ejecutar funciones con efectos secundarios (por ejemplo imprimir), pero **no** es la mejor práctica si el propósito es solo efectos secundarios: un bucle `for` es más claro.


In [ ]:
def imprimir(x):
    print('->', x)

list(map(imprimir, [1, 2, 3]))  # fuerza la evaluación

# Mejor con for:
for x in [1, 2, 3]:
    imprimir(x)


## 7. Uso con `operator` y `methodcaller` — llamadas a métodos/atributos de forma eficiente


In [ ]:
from operator import methodcaller, itemgetter

strings = [' hello ', ' world ', 'python ']
# Aplicar strip con methodcaller
print(list(map(methodcaller('strip'), strings)))

tuplas = [(1, 'a'), (2, 'b'), (3, 'c')]
print(list(map(itemgetter(1), tuplas)))  # extraer segundo elemento de cada tupla


## 8. `functools.partial` para prellenar argumentos


In [ ]:
from functools import partial

def potencia(base, exp):
    return base ** exp

cuadrado = partial(potencia, exp=2)
print(list(map(cuadrado, [2, 3, 4])))


## 9. Combinar `map()` con `itertools` para pipelines eficientes


In [ ]:
import itertools

data = range(1, 100)
# pipeline: map -> filter -> islice (solo los primeros 5 resultados válidos)
def es_par(x):
    return x % 2 == 0

it = map(lambda x: x * x, data)
it = filter(es_par, it)
it = itertools.islice(it, 5)
print(list(it))


## 10. Errores comunes y casos límite

- Usar `map` con una función que no acepta el número correcto de argumentos -> `TypeError`.
- `map` termina con el iterable más corto cuando hay varios.
- En Python 3 `map(None, ...)` no tiene sentido (a diferencia de Python 2).


In [ ]:
try:
    # función que espera 1 arg pero le pasamos 2 iterables
    list(map(lambda x: x * 2, [1,2], [3,4]))
except TypeError as e:
    print('TypeError esperado:', e)

# map se detiene con el iterable más corto
print(list(map(lambda x, y: (x, y), [1,2,3], [10,20])))


## 11. `map()` vs `for` y cuándo preferir cada uno

- Usa `map()` cuando:
  - Tienes una función existente y quieres aplicarla a cada elemento.
  - Necesitas un iterador perezoso en una pipeline.
- Usa `for` cuando:
  - Hay efectos secundarios claros o la transformación es compleja y deseas legibilidad.


## 12. Rendimiento (nota rápida)

- `map()` puede ser ligeramente más rápido que una list comprehension cuando la función aplicada es una función ya compilada (no `lambda`) porque evita la sobrecarga del bucle en Python puro en algunos casos.
- Pero la diferencia es pequeña; favorece claridad.


In [ ]:
import timeit

setup = 'nums = list(range(1000))\nfrom math import sqrt'
stmt_map = 'list(map(sqrt, nums))'
stmt_comp = '[sqrt(x) for x in nums]'
t_map = timeit.timeit(stmt_map, setup=setup, number=100)
t_comp = timeit.timeit(stmt_comp, setup=setup, number=100)
print('map:', t_map)
print('list comp:', t_comp)


## 13. Casos prácticos (mini-problemas resueltos)

1. Convertir lista de fechas en strings a objetos `datetime`.
2. Normalizar (lowercase + strip) una lista de palabras.
3. Aplicar función con múltiples iterables (suma ponderada).


In [ ]:
from datetime import datetime

# 1: parsear fechas
dates = ['2020-01-01', '2021-12-31', ' 2022-06-15 ']
parsed = list(map(lambda s: datetime.strptime(s.strip(), '%Y-%m-%d'), dates))
print(parsed)

# 2: normalizar palabras
words = [' Hello', 'WORLD ', ' PyThOn ']
print(list(map(lambda s: s.strip().lower(), words)))

# 3: suma ponderada
vals = [10, 20, 30]
weights = [0.1, 0.2, 0.7]
print(list(map(lambda v, w: v * w, vals, weights)))


## 14. Buenas prácticas y checklist para una competencia

- Prefiere funciones nombradas cuando la lógica es compleja (mejor trazabilidad en trazas y perfiles).
- Si necesitas todos los resultados en memoria, conviértelo a `list()` al final.
- Evita `map()` para efectos secundarios: usa `for`.
- Cuando uses múltiples iterables, confirma longitudes o usa `itertools.zip_longest` si quieres otro comportamiento.
- Comenta tu intención: `# pipeline: map(...) -> filter(...)` ayuda a los revisores.


## 15. Apéndice — ejemplos avanzados

- `zip_longest` para emular comportamiento diferente al detenerse en el más corto.
- `map()` con métodos de clase a través de `operator.methodcaller`.


In [ ]:
from itertools import zip_longest

a = [1,2]
b = [10,20,30]
print('map (se corta):', list(map(lambda x, y: (x,y), a, b)))
print('zip_longest + map emulado:', list(map(lambda x_y: x_y, zip_longest(a, b, fillvalue=None))))


### Fin

Este notebook está pensado para cubrir casi todos los casos de uso razonables de `map()` en Python 3. Si quieres, puedo añadir:
- Más benchmarks (big-O, memoria) comparativos.
- Sección para `multiprocessing.Pool.map` y `concurrent.futures` (paralelismo).
- Transformaciones aplicadas a `pandas.Series.map` vs `map()` built-in.

Dime si deseas que agregue alguna de estas secciones y la incorporo.